In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
#Carregando o dataset
df = pd.read_excel("../dataset/dry_bean_dataset.xls")

In [8]:
#Normalização das variáveis
ss = StandardScaler()
df_n = df.Area, df.Perimeter, df.MajorAxisLength, df.MinorAxisLength, df.Eccentricity, df.ConvexArea, df.EquivDiameter
df_n = ss.fit_transform(df_n)

#Inserindo os dados normalizados no dataframe original
df["Area"] = df_n[0]
df["Perimeter"] = df_n[1]
df["MajorAxisLength"] = df_n[2]
df["MinorAxisLength"] = df_n[3]
df["Eccentricity"] = df_n[4]
df["ConvexArea"] = df_n[5]
df["EquivDiameter"] = df_n[6]

#Dividindo a base em treino e teste
X = df.drop("Class", axis=1)
y = pd.get_dummies(df["Class"])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=123)